In [1]:
import numpy as np
import json
import pickle
import random
import nltk
import datetime
import wikipedia
import pywhatkit

from nltk.stem import WordNetLemmatizer

from tensorflow.keras.models import load_model

lemmatizer = WordNetLemmatizer()
intents = json.loads(open('intents.json').read())

words = pickle.load(open('words.pkl', 'rb'))
classes = pickle.load(open('classes.pkl', 'rb'))
model = load_model('chatbot_model.h5')


In [3]:
import speech_recognition as sr
import pyttsx3

listener = sr.Recognizer()
engine = pyttsx3.init()
voices = engine.getProperty('voices')
engine.setProperty('voice',voices[1].id)
engine.setProperty("rate", 190)

def talk(text):
    engine.say(text)
    engine.runAndWait()
    
def listen():
    try:
        with sr.Microphone() as source:
            print('Listening....\n')
            voice = listener.listen(source)
            command = listener.recognize_google(voice)
            command = command.lower()
            return command
    
    except:
        pass


In [ ]:

def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word) for word in sentence_words]
    return sentence_words

def bag_of_words(sentence):
    sentence_words = clean_up_sentence(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

def predict_class(sentence):
    bow = bag_of_words(sentence)
    res = model.predict(np.array([bow]))[0]
    ERROR_THRESHOLD =0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    for r in results:
        return_list.append({'intent':classes[r[0]], 'probability': str(r[1])})
    return return_list

def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            result = random.choice(i['responses'])
            break
    print("Bot : "+result)
    talk(result)
    return tag

print('Begin...\n')

while True:
    message = listen()
    if message:
        ints = predict_class(message)
        tag = get_response(ints, intents)
    if tag == "goodbye":
        break
    elif tag == "google":
        pywhatkit.search(message)
    elif tag == "youtube":
        pywhatkit.playonyt(message)
    elif tag == "wikipedia":
        info = wikipedia.summary(message, 1)
        print(info)
        talk(info)
    elif tag == "time":
        time = datetime.datetime.now().strftime('%I:%M %p')
        print('Current time is ' + time)
        talk('Current time is ' + time)
    elif tag == "date":
        date = datetime.datetime.now().strftime('%d-%B-%Y')
        print('Current date is ' + date)
        talk('Current date is ' + date)
    elif tag == "day":
        day = datetime.datetime.now().strftime('%A')
        print(day)
        talk(day)
        

Begin...

hi
Bot : Hello!
hi
Bot : Hello!
hi
Bot : Hello!
whats ur name
Bot : Hey, How can I help?
who is donald trump
Bot : 
